# Importações das bibliotecas externas

In [ ]:
import numpy as np
import queue
from collections import deque
import time
import math
import matplotlib.pyplot as plt
from sklearn.neighbors import NearestNeighbors
import numpy as np
from sklearn.neighbors import NearestNeighbors
import random
import math
from heapq import heappop, heappush

# Definições globais de variável

In [ ]:
seed = 13
num_points = 2000

executionsK3 = [
    [(9488, 5880), (8564, 9598)],
    [(6094, 5922), (7183, 5343)],
    [(2881, 2367), (3722, 6447)],
    [(8536, 3285), (8428, 3559)],
    [(4710, 1963), (5792, 4198)],
    [(1945, 141), (3712, 5834)],
    [(8603, 6388), (7864, 9658)],
    [(3320, 2774), (3876, 3771)],
    [(3320, 2774), (3876, 3771)],
    [(5624, 2898), (4458, 2414)]
]

executionsK7 = [
    [(4500, 6121), (7195, 1604)],
    [(4000, 4408), (91, 588)],
    [(8037, 9967), (377, 5124)],
    [(4993, 3921), (7960, 1856)],
    [(3531, 2629), (8690, 4984)],
    [(38, 9841), (4219, 9099)],
    [(38, 9841), (4219, 9099)],
    [(127, 2626), (4365, 3708)],
    [(9927, 3093), (2894, 2180)],
    [(3847, 9224), (9829, 4439)]
]


executionsK11 = [
    [(447, 130), (1490, 7028)],
    [(1427, 7307), (1521, 7211)],
    [(8178, 672), (4, 8961)],
    [(121, 9000), (142, 440)],
    [(8563, 39), (516, 9755)],
    [(121, 9000), (8563, 39)],
    [(9920, 6004), (5060, 8442)],
    [(8464, 1707), (1326, 3148)],
    [(3741, 3894), (3756, 4684)],
    [(3756, 4684), (4270, 500)]
]


# Geração da rede KNN

In [ ]:
def gererateGraph(k):
  np.random.seed(13)
  new_points = np.random.randint(0, 10000, size=(num_points, 2))


  # Calculando os vizinhos mais próximos usando o algoritmo KNN
  knn = NearestNeighbors(n_neighbors=k+1)
  knn.fit(new_points)
  distances, indices = knn.kneighbors(new_points)

  # Criando o grafo no formato desejado
  graph = {}

  for i in range(len(new_points)):
      vertex = tuple(new_points[i])
      neighbors = []

      for j in indices[i]:
          if i != j:  # Evita criar uma aresta entre um vértice e ele mesmo
              neighbors.append(tuple(new_points[j]))

      graph[vertex] = neighbors
  return graph

# BFS

In [ ]:

def bfs(graph, start, end):
    queue = deque([(start, [start])])
    visited = set([start])
    paths = []
    count = 0

    while queue:
        node, path = queue.popleft()
        count = count + 1
        if node == end:
            paths.append((path))
            paths.append(list(queue))
            return paths, count
          

        for neighbor in graph[node]:
            if neighbor not in visited:                        # se o filho do nó atual não foi visitado, add ao fim da fila.
                queue.append((neighbor, path + [neighbor]))    # faz isso pra todos os filhos F->A,B,C depois (F,A), (F,B), (F,C), dna prox rodada add ao FIM: (F,A,H)
                visited.add(neighbor)                          # depois pergunta se já chegou.  Então queue possui todos os filhos de um dado nível além da resposta final
                
    return None, count

def dfs(graph, start, end):
    stack = [(start, [start])]
    visited = set([start])
    all_paths = []
    count = 0

    while stack:
        node, path = stack.pop()
        count = count + 1
        
        if node == end:
            all_paths.append((path))
            all_paths.append(list(stack))
            return all_paths, count

        for neighbor in graph[node]:
            if neighbor not in visited:
                stack.append((neighbor, path + [neighbor]))
                visited.add(neighbor)

    return None, count

def calculate_path_distance(path):
    total_distance = 0

    for i in range(len(path) - 1):
        current_point = path[i]
        next_point = path[i + 1]

        distance = math.sqrt((next_point[0] - current_point[0])**2 + (next_point[1] - current_point[1])**2)
        total_distance += distance

    return total_distance

def plot_graph_path_BFS_DFS(graph, paths):
    fig, ax = plt.subplots(figsize=(24, 18))
    ax.set_aspect('equal', adjustable='datalim')

    # Plotar as arestas do grafo
    for node in graph:
      x, y = node
      # ax.text(x, y, f'({x},{y})', color='black')
      for neighbor in graph[node]:
          nx, ny = neighbor
          ax.plot([x, nx], [y, ny], color='gray')

    # Plotar os demais caminhos em amarelo
    for path in paths[1:]:
      for visited in path:
          # color = random.choice(['yellow', 'green', 'blue', 'orange', 'pink'])
          for i in range(len(visited[1]) - 1): # A STAR É 2, BFS E DFS EH 1
              node1 = visited[1][i]
              node2 = visited[1][i + 1]
              x = [node1[0], node2[0]]
              y = [node1[1], node2[1]]
              ax.plot(x, y, color='blue')
              ax.plot(x, y, 'o', color='blue', markersize=5)

    # Plotar o primeiro caminho em vermelho
    first_path = paths[0]
    for i in range(len(first_path) - 1):
        x = [first_path[i][0], first_path[i+1][0]]
        y = [first_path[i][1], first_path[i+1][1]]
        ax.plot(x, y, color='red')  # aresta
        ax.text(first_path[i][0], first_path[i][1], f'({first_path[i][0]},{first_path[i][1]})', color='black') # coordenada
        ax.text(first_path[i+1][0], first_path[i+1][1], f'({first_path[i+1][0]},{first_path[i+1][1]})', color='black') # coordenada
        if i + 1 == len(first_path):
          ax.plot(x, y, '^', color='red', markersize=3) # triangulo - destino
        else:
          ax.plot(x, y, 'o', color='red', markersize=3) # ponto

    ax.set_xlim([0, 10000])
    ax.set_ylim([0, 10000])



# Escrita no sheets

In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

def escreveNaTabela(results):
  spreadsheet = gc.open('Execuções Algoritmos de Busca')
  sheet = spreadsheet.worksheet('Execuções')

  for row in results:
    sheet.append_row(row)

# Execuções

In [ ]:
def executeSearch(search, executions, k):

  graph = gererateGraph(k)
  results = []
  for i, execution in enumerate(executions):
    start_time = time.time()
    if search == 'bfs':
      path, count_visited = bfs(graph, execution[0], execution[1])
    else:
      path, count_visited = dfs(graph, execution[0], execution[1])

    elapsed_time = time.time() - start_time

    # plot_graph(graph, execution)
    if path is not None:

        results.append([
            str(i+1), #nro execução
            str(search), # sigla algoritmo de busca
            str(k), # quantidade de vizinhos
            str(seed), # seed aleatoria
            str(execution[0]), # nó origem
            str(execution[1]), # nó destino
            str(round(elapsed_time, 10)), # tempo gasto
            str(round(calculate_path_distance(path[0]), 3)), #distancia euclidiana
            str(len(path[0])), # nro de nós até chegar ao destino
            str(count_visited) # nro de nós visitados no total
        ])

        # plot_graph_path_BFS_DFS(graph, path)
    else:
        print("Não há caminho alcançável entre os nós de origem e destino.")
  escreveNaTabela(results)



In [ ]:
executeSearch('dfs', executionsK3, 3)
executeSearch('dfs', executionsK7, 7)
executeSearch('dfs', executionsK11, 11)

executeSearch('bfs', executionsK3, 3)
executeSearch('bfs', executionsK7, 7)
executeSearch('bfs', executionsK11, 11)
